In [1]:

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# function to create a folder to store the results if it does not exist


In [2]:
def ResultsFolder(folder_path):
    """ Ensure that a directory exists at the given path, and if not, create it. """
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
        print(f"Directory created: {folder_path}")
    else:
        print(f"Directory already exists: {folder_path}")

# obtain the file name of the flood event(.csv)

rawdata_folder = 'C:/Users/mgalib/Box/Finley_UH_work/Data/TN/rainfall-runoff-events_TN/'
direct = 'C:/Users/mgalib/Box/Finley_UH_work/Data/TN'
#folder_path = r'.\raw_data'
filename = os.listdir(rawdata_folder)

# create a dataframe to store PRF of all the runoff events

PRF = pd.DataFrame(0, index=filename,columns=['PRF'])
PRF.index.name = 'File_Name'

for file in filename:
    rawdata = pd.read_csv(rawdata_folder + file)

    print("\nResults of the runoff event in " + file + ":\n")

    file = file.split(".")[0]

    # Assuming 'direct' is defined somewhere above this snippet as your base directory path
    Folder0 = os.path.join(direct, 'UH')
    ResultsFolder(Folder0)  # Ensure this folder exists or create it

   
    Folder1 = os.path.join(direct, 'UH', file)
    ResultsFolder(Folder1)  # Ensure this folder exists or create it

        # obtain the drainage area of the watershed
    # the value of area is stored in the second row and third column in the original file

    # Correctly accessing the 'Area' column for the first row
    area = rawdata.loc[0, 'Area']
   
    print("The drainage area of the watershed is " + str(area) + " mile\u00b2.\n".format())

    # compute the time interval (in seconds) of the discharge data

    rawdata.iloc[:,0] = pd.to_datetime(rawdata.iloc[:,0])
    delta_t = (rawdata.iloc[1,0] - rawdata.iloc[0,0]).seconds
    print("The time interval of the discharge data is " + str(delta_t/60) + " mins.\n")

    # create a new time index in days

    index = np.linspace(0,delta_t/86400*(len(rawdata)-1),len(rawdata))

    # compute the direct runoff(cfs)
    # separate baseflow with straight line method and assign the first value as baseflow

    baseflow = rawdata.iloc[0,1]

    directQ_cfs = pd.DataFrame(0.0, index=index, columns=['Direct_runoff_cfs'])

    for i in range(len(rawdata)):
        if rawdata.iloc[i,1] - baseflow < 0:
            directQ_cfs.iloc[i] = 0
        else:
            directQ_cfs.iloc[i] = rawdata.iloc[i,1] - baseflow   

    # compute the depth of the direct runoff(in)

    directQ_in = pd.DataFrame(0.0, index=index, columns=['Direct_runoff_in'])

    for i in range(1,len(rawdata)):
        # 1 mile = 5280 ft, and 1 ft = 12 in
        directQ_in.iloc[i] = float((directQ_cfs.iloc[i-1]+directQ_cfs.iloc[i])/2*delta_t*12/(area*5280**2))

    # derive the Unit Hydrograph(cfs/in), save as CSV & JPEG in the "UH" folder

    sum_directQ_in = float(directQ_in.iloc[:,0].sum())

    UH = pd.DataFrame(0.0, index=index, columns=['UH_cfs/in'])

    for i in range(len(rawdata)):
        UH.iloc[i] = float(directQ_cfs.iloc[i]/sum_directQ_in)

    UH.index.name = 'Days'

    UH.to_csv(Folder1 + "Unit Hydrograph of " + file + ".csv")

    UH.plot(figsize=(12,6))
    plt.title("Unit Hydrograph of "+ file)
    plt.legend(["Discharge"], loc='best',edgecolor='k')
    plt.xlabel("Days")
    plt.ylabel("Discharge (cfs/in)")
    plt.savefig(Folder1 + "Unit Hydrograph of " + file + ".jpeg")
    plt.close()

    # check whether the depth of the UH is equal to 1 inch

    depth_UH = UH.iloc[:,0].sum()*delta_t*12/(area*5280**2)
    depth_UH = round(depth_UH,1)

    print("The depth of direct runoff in the Unit Hydrograph is " + str(depth_UH) + " inch.")

    if depth_UH != 1.0:
        print("Note: The ordinates of UH shall be adjusted by proportion so that the depth of direct runoff is 1 inch!\n")

    print("- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -")
    
    # generate the Dimensionless Unit Hydrograph, save as CSV & JPEG in the "UH" folder

    Q_peak = UH.iloc[:,0].max()
    T_peak = UH[UH.iloc[:,0]==Q_peak].index[0]

    index_dl = index/T_peak

    DUH = pd.DataFrame(0.0, index=index_dl, columns=['Q/Qp'])

    for i in range(len(rawdata)):
        DUH.iloc[i] = float(UH.iloc[i]/Q_peak)

    DUH.index.name = 't/Tp'

    DUH.to_csv(Folder1 + "Dimensionless Unit Hydrograph of " + file + ".csv")

    DUH.plot(figsize=(12,6))
    plt.title("Dimensionless Unit Hydrograph of " + file)
    plt.legend(["DUH"], loc='best',edgecolor='k')
    plt.xlabel("t / Tp")
    plt.ylabel("Q / Qp")
    plt.savefig(Folder1 + "Dimensionless Unit Hydrograph of " + file + ".jpeg")
    plt.close()

    # compute PRF based on the UH

    PRF.loc[file+".csv"] = round(float(Q_peak*T_peak*24/area))

# save PRF of all the runoff events as CSV in the "UH" folder

csv_file_path = os.path.join(Folder0, "PRF of Watersheds.csv")
PRF.to_csv(csv_file_path)

print("UH derivation for the " + str(len(filename)) + " runoff events is done!")



Results of the runoff event in Event_3426385_2000-03-28.csv:

Directory created: C:/Users/mgalib/Box/Finley_UH_work/Data/TN\UH
Directory created: C:/Users/mgalib/Box/Finley_UH_work/Data/TN\UH\Event_3426385_2000-03-28
The drainage area of the watershed is 19.85857201 mile².

The time interval of the discharge data is 195.0 mins.



C:\Users\mgalib\AppData\Local\Temp\ipykernel_13792\2227566579.py:73: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  directQ_in.iloc[i] = float((directQ_cfs.iloc[i-1]+directQ_cfs.iloc[i])/2*delta_t*12/(area*5280**2))
C:\Users\mgalib\AppData\Local\Temp\ipykernel_13792\2227566579.py:82: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  UH.iloc[i] = float(directQ_cfs.iloc[i]/sum_directQ_in)


The depth of direct runoff in the Unit Hydrograph is 1.0 inch.
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

Results of the runoff event in Event_3426385_2000-04-01.csv:

Directory already exists: C:/Users/mgalib/Box/Finley_UH_work/Data/TN\UH
Directory created: C:/Users/mgalib/Box/Finley_UH_work/Data/TN\UH\Event_3426385_2000-04-01
The drainage area of the watershed is 19.85857201 mile².

The time interval of the discharge data is 1380.0 mins.



C:\Users\mgalib\AppData\Local\Temp\ipykernel_13792\2227566579.py:118: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  DUH.iloc[i] = float(UH.iloc[i]/Q_peak)
C:\Users\mgalib\AppData\Local\Temp\ipykernel_13792\2227566579.py:73: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  directQ_in.iloc[i] = float((directQ_cfs.iloc[i-1]+directQ_cfs.iloc[i])/2*delta_t*12/(area*5280**2))
C:\Users\mgalib\AppData\Local\Temp\ipykernel_13792\2227566579.py:82: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  UH.iloc[i] = float(directQ_cfs.iloc[i]/sum_directQ_in)
C:\Users\mgalib\AppData\Local\Temp\ipykernel_13792\2227566579.py:118: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeEr

The depth of direct runoff in the Unit Hydrograph is 1.0 inch.
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

Results of the runoff event in Event_3426385_2007-03-28.csv:

Directory already exists: C:/Users/mgalib/Box/Finley_UH_work/Data/TN\UH
Directory created: C:/Users/mgalib/Box/Finley_UH_work/Data/TN\UH\Event_3426385_2007-03-28
The drainage area of the watershed is 19.85857201 mile².

The time interval of the discharge data is 450.0 mins.



...

C:\Users\mgalib\AppData\Local\Temp\ipykernel_13792\2227566579.py:118: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  DUH.iloc[i] = float(UH.iloc[i]/Q_peak)



Results of the runoff event in Event_3430147_2004-03-22.csv:

Directory already exists: C:/Users/mgalib/Box/Finley_UH_work/Data/TN\UH
Directory created: C:/Users/mgalib/Box/Finley_UH_work/Data/TN\UH\Event_3430147_2004-03-22
The drainage area of the watershed is 20.89073166 mile².

The time interval of the discharge data is 15.0 mins.



C:\Users\mgalib\AppData\Local\Temp\ipykernel_13792\2227566579.py:73: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  directQ_in.iloc[i] = float((directQ_cfs.iloc[i-1]+directQ_cfs.iloc[i])/2*delta_t*12/(area*5280**2))
C:\Users\mgalib\AppData\Local\Temp\ipykernel_13792\2227566579.py:82: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  UH.iloc[i] = float(directQ_cfs.iloc[i]/sum_directQ_in)


The depth of direct runoff in the Unit Hydrograph is 0.0 inch.
Note: The ordinates of UH shall be adjusted by proportion so that the depth of direct runoff is 1 inch!

- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -


IndexError: index 0 is out of bounds for axis 0 with size 0